<a href="https://colab.research.google.com/github/Tower5954/TensorFlow/blob/main/Natural_Language_Processing_with_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to NLP fundamentals in TensorFlow

NLP has the goal of deriving information out of natural language (could be sequences of text or speech).

Another common term for NLP problems is sequence to sequence problems (seq2seq).

### Check GPU

In [1]:
!nvidia-smi -L

GPU 0: Tesla K80 (UUID: GPU-4b473273-72e9-a973-c120-451867203037)


### Get helper-functions

In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2022-03-20 10:17:35--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-03-20 10:17:35 (39.0 MB/s) - ‘helper_functions.py’ saved [10246/10246]



#### Import a series of helper functions for the notebook

In [3]:
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

### Get a text dataset

The dataset that we are going to be using is Kaggle's introduction to NLP dataset (text samples of tweetslabelled as 'Disaster' or 'Not Disaster').

See the original source here: https://www.kaggle.com/c/nlp-getting-started/data

In [4]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

# Unzip the data
unzip_data("nlp_getting_started.zip")

--2022-03-20 10:17:41--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.136.128, 142.250.148.128, 209.85.200.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.136.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.008s  

2022-03-20 10:17:41 (76.0 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Visualising a text dataset

To visualise our text samples, we first have to read them in, one way is to use Python: https://realpython.com/read-write-files-python/

Another way to do this is to use Pandas...

In [5]:
import pandas as pd

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
train_df["text"][0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [7]:
train_df["text"][1]

'Forest fire near La Ronge Sask. Canada'

In [8]:
# Shuffle training dataset

train_df_shuffle = train_df.sample(frac=1, random_state=42)
train_df_shuffle.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [9]:
# What does the test dataframe look like?
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [10]:
# How many examples of each class?
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [11]:
# How many total samples?
len(train_df), len(test_df)

(7613, 3263)

In [12]:
# Visualise some random training examples
import random

random_index = random.randint(0, len(train_df)-5) # Create random indexes not higher than the total number of samples 
for row in train_df_shuffle[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
  print(f"Text:\n{text}\n")
  print("---\n")

Target: 0 (not real disaster)
Text:
@SenSchumer Is this what U want Netanyahu leading these UNITED STATES into a CATASTROPHIC religious world war? ENOUGH already!

---

Target: 1 (real disaster)
Text:
Japan marks 70th anniversary of Hiroshima atomic bombing: Bells tolled in Hiroshima on Thursday as Japan marked 70Û_ http://t.co/NBZiKcJpHp

---

Target: 1 (real disaster)
Text:
Udhampur terror attack: Militants attack police post 2 SPOs injured: Suspected militants tonight attacked a p... http://t.co/cEKbxJmPBj

---

Target: 0 (not real disaster)
Text:
@Welles_7 he was injured. He is a pro bowl back.

---

Target: 0 (not real disaster)
Text:
Hollywood movie about trapped miners released in #Chile http://t.co/r18aUtnLSd #ZippedNews http://t.co/CNqaE9foj6

---



### Split data into training and validation sets


Since the test set has no labels and we need a way to evalaute our trained models, we'll split off some of the training data and create a validation set.

When our model trains (tries patterns in the Tweet samples), it'll only see data from the training set and we can see how it performs on unseen data using the validation set.

We'll convert our splits from pandas Series datatypes to lists of strings (for the text) and lists of ints (for the labels) for ease of use later.

To split our training dataset and create a validation dataset, we'll use Scikit-Learn's `train_test_split()` method and dedicate 10% of the training samples to the validation set.

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
# Use train_test_split to split training data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffle["text"].to_numpy(),
                                                                            train_df_shuffle["target"].to_numpy(),
                                                                            test_size=0.1, # Use 10% of training data for validation split
                                                                            random_state=42) 

In [15]:
# Check the lengths
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [16]:
# Check the first 10 samples 
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), array([0, 

## Converting text into numbers

* Tokenisation:
  * Straight mapping from token to number (can be modelled but quickly gets too big).

* Embedding:
  * Richer representation of relationships between tokens (can limit size + can be learned)


In NLP, there are two main concepts for turning text into numbers:

  * **Tokenization** - A straight mapping from word or character or sub-word to a numerical value. There are three main levels of tokenization:

1. Using **word-level tokenization** with the sentence "I love TensorFlow" might result in "I" being `0`, "love" being `1` and "TensorFlow" being `2`. In this case, every word in a sequence considered a single **token**.
2. **Character-level tokenization**, such as converting the letters A-Z to values `1-26`. In this case, every character in a sequence considered a single **token**.
3. **Sub-word tokenization** is in between word-level and character-level tokenization. It involves breaking invidual words into smaller parts and then converting those smaller parts into numbers. For example, "my favourite food is pineapple pizza" might become "my, fav, avour, rite, fo, oo, od, is, pin, ine, app, le, piz, za". After doing this, these sub-words would then be mapped to a numerical value. In this case, every word could be considered multiple **tokens**.

* **Embeddings** - An embedding is a representation of natural language which can be learned. Representation comes in the form of a feature vector. For example, the word "dance" could be represented by the 5-dimensional vector `[-0.8547, 0.4559, -0.3332, 0.9877, 0.1112]`. It's important to note here, the size of the **feature vector** is tuneable. There are two ways to use embeddings:
1. **Create your own embedding** - Once your text has been turned into numbers (required for an embedding), you can put them through an embedding layer (such as `tf.keras.layers.Embedding`) and an embedding representation will be learned during model training.
2. **Reuse a pre-learned embedding** - Many pre-trained embeddings exist online. These pre-trained embeddings have often been learned on large corpuses of text (such as all of Wikipedia) and thus have a good underlying representation of natural language. You can use a pre-trained embedding to initialize your model and fine-tune it to your own specific task.

In [17]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [18]:
# Use the default TextVectorization parameters

text_vectoriser = TextVectorization(max_tokens=10000, # how mant words in the vocabulary
                                    standardize="lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams=None, # create groups of n-words?
                                    output_mode="int", # how to map tokens to numbers
                                    output_sequence_length=None, # how long do you want your sequences to be?
                                    pad_to_max_tokens = True)

In [19]:
# Find the average number of tokens (words) in the training tweets
train_sentences[0].split()

['@mogacola', '@zamtriossu', 'i', 'screamed', 'after', 'hitting', 'tweet']

In [20]:
len(train_sentences[0].split())

7

In [21]:
round(sum([len(i.split()) for i in train_sentences]))

102087

In [22]:
round(sum([len(i.split()) for i in train_sentences]) / len(train_sentences))

15

In [23]:
# Set up a text vectorization variables

max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15 # max length our sequences will be (e.g. how many words from a tweet does our model see)

text_vectoriser = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [24]:
# Fit the text vectoriser to the training text
text_vectoriser.adapt(train_sentences)

In [25]:
# Create a sample sentence and tokenise it
sample_sentence = "There's a flood in my street!"
text_vectoriser([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [26]:
# Choose a random sentence from the training dataset and tokenising it
random_sentence = random.choice(train_sentences)

In [27]:
print(f"Original text:\n {random_sentence}\n\n Vectorised version:")
text_vectoriser([random_sentence])


Original text:
 California Bush fires please evacuate affected areas ASAP when california govts advised you to do so http://t.co/ubVEVUuAch

 Vectorised version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[  90,  657,  109,  170,  279,  446, 1903, 2554,   45,   90,    1,
        6257,   12,    5,   68]])>

In [28]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectoriser.get_vocabulary() # get all the unique words in our training data
top_5_words = words_in_vocab[:5] # get the most common words
bottom_5_words = words_in_vocab[-5:] # get the least common words
print(f"No of words in vocab: {len(words_in_vocab)}")
print(f"Most common words used: {top_5_words}")
print(f"Least common words used: {bottom_5_words}")

No of words in vocab: 10000
Most common words used: ['', '[UNK]', 'the', 'a', 'in']
Least common words used: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


### Creating an Embedding using an Embedding Layer


To make the embedding we are going to use TensorFlow's embedding layer:
https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding


The parameters we care most about for our embedding layer:
* `input_dim` = the size of our vocabulary.
* `output_dim` = the size of the output embedding vector, for example, a value of 100 would mean each token getting represented by a vector 100 long.
* `input_length` = length of sequences being passed to the embedding layer.  

In [29]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length, # set input shape
                             output_dim=128, # output shape # NN work best with numbers divisible by 8 
                             embeddings_initializer="uniform", # default
                             input_length=max_length # how long is each input
                             )

embedding

In [30]:
# Choose a random sentence from the training set
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\n\n Embedded version:")

# Embed the random sentence (turn it into dense vectors of fixed sizes)
sample_embed = embedding(text_vectoriser([random_sentence]))
sample_embed



Original text:
 St steel coffee cafetiere exploded this am with loud bang hot coffee &amp; grounds shot over table clean crockery phone
tablet. How?

 Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 8.16583633e-05,  4.44285758e-02,  3.76596935e-02, ...,
          3.01312320e-02,  2.74854787e-02,  9.05774534e-04],
        [-2.48764828e-03,  1.20739825e-02,  4.20223586e-02, ...,
          2.23073252e-02,  2.57867239e-02,  2.93162502e-02],
        [-1.88018084e-02,  3.40879001e-02, -4.44382429e-03, ...,
          3.20738293e-02,  2.31709145e-02, -1.28142722e-02],
        ...,
        [-2.53428575e-02, -9.40956175e-04, -9.14417580e-03, ...,
          8.69470835e-03,  2.53981389e-02, -2.65178438e-02],
        [-2.28965767e-02, -1.10452883e-02,  2.37836875e-02, ...,
         -2.89625060e-02, -1.43787488e-02, -3.95053737e-02],
        [ 2.85957567e-02, -5.01025841e-03,  4.21635769e-02, ...,
          3.23884152e-02, -1.82166100e-02, -4.91735451e-02]]],
      dtype=float32)>

In [31]:
# Check out a single token's embedding
sample_embed[0][0], sample_embed[0][0].shape, random_sentence


(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 8.1658363e-05,  4.4428576e-02,  3.7659694e-02,  1.2553755e-02,
         4.2832557e-02, -2.9218758e-02,  2.8948341e-02,  3.0899670e-02,
        -4.7180224e-02,  4.0032975e-03, -1.5256118e-02, -3.2475937e-02,
         1.2159742e-02,  8.7733641e-03, -4.9171280e-02,  8.7304004e-03,
         3.9577711e-02,  4.6788204e-02,  1.6019825e-02, -8.9787953e-03,
        -2.5726533e-02,  1.0554992e-02,  1.7445695e-02,  4.5322403e-03,
        -1.0448895e-02,  7.7735409e-03,  4.2967785e-02, -4.7720399e-02,
        -8.1694722e-03, -1.1487685e-02, -5.9650056e-03,  3.8521539e-02,
        -1.3253607e-02,  3.4814548e-02,  3.6709879e-02, -1.8039167e-02,
        -2.7213681e-02, -1.8336020e-02,  4.5557145e-02, -2.0460665e-02,
        -3.9591588e-02, -5.8693774e-03, -2.5477111e-02,  1.3136517e-02,
        -6.0562976e-03,  5.5652149e-03, -4.5236599e-02,  2.6013143e-03,
        -4.7791231e-02,  1.9456577e-02,  1.8237364e-02,  1.5640941e-02,
         1.5649

## Modelling a text dataset (running a series of numbers).

We have now got a way of turning text sequences into numbers, it is time to start building a series of modelling experiments.

We will start with a baseline and move on from there.

* Model 0: Naive Bayes (baseline), from the [Sklearn ML map.](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html)
* Model 1: Feed-forward neural network network (dense model).
* Model 2: LSTM model (RNN). 
* Model 3: GRU model (RNN).
* Model 4: Biderectional-LSTM model (RNN).
* Model 5: 1D Convolutional Neural Network (CNN).
* Model 6: TensorFlow Hub pretrained feature extractor (using transfer learning for NLP)
* Model 7: The same as model 6 with 10% of the training data.


Use the standard steps in modelling with TensorFlow:
* Create a model.
* Build a model.
* Fit a model.
* Evaluate the model.

### Model 0: Getting a baseline.

As with all machine learning modelling experiments, it's important to create a baseline model so you've got a benchmark for future experiments to build upon.

To create the baseline, we will use Sklearn's Multinomial Naive Bayes using the TF-IDF formula to convert our words to numbers.

> **Note** It is common practice to use non-DL algorithms as a baseline because of their speed and then later using DL to see if you can improve upon performance. 